In [21]:
# === Bootstrap cell: shared config, paths, JSON I/O, and GitHub REST/GraphQL helpers ===

import os, json, time, re, random, pathlib
from typing import Dict, Any, Optional, Tuple, List
from datetime import datetime, timezone

import requests
import yaml
from dotenv import load_dotenv

# -------------------------------------------------------------------
# Locate repo root reliably (works no matter where you open the .ipynb)
# -------------------------------------------------------------------
def _find_repo_root(start: pathlib.Path) -> pathlib.Path:
    cur = start.resolve()
    for _ in range(8):  # walk up to 8 levels
        if (cur / "config" / "config.yaml").exists():
            return cur
        cur = cur.parent
    raise FileNotFoundError("Couldn't locate repo root (no config/config.yaml found upward).")

REPO_ROOT = _find_repo_root(pathlib.Path.cwd())
CONFIG_PATH = REPO_ROOT / "config" / "config.yaml"

# -------------------------------------------------------------------
# Load config + token
# -------------------------------------------------------------------
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    CFG: Dict[str, Any] = yaml.safe_load(f) or {}

load_dotenv(REPO_ROOT / ".env")  # local only; do not commit .env
TOKEN_ENV = (CFG.get("github_token_env") or "GITHUB_TOKEN").strip()
GITHUB_TOKEN = os.getenv(TOKEN_ENV, "").strip()
if not GITHUB_TOKEN:
    raise RuntimeError(f"Missing token in environment variable {TOKEN_ENV}. "
                       f"Create .env from .env.example and set {TOKEN_ENV}=...")

# Repo + output root
REPO = CFG["repo"]  # "owner/name"
OWNER, NAME = REPO.split("/", 1)
OUT_ROOT = pathlib.Path(CFG.get("out_root") or NAME)  # "<name>" if null

# Behavior
OVERWRITE = bool(CFG.get("overwrite", True))
VERBOSE = bool(CFG.get("verbose_logs", True))

# Networking knobs
REQ_TIMEOUT = int(CFG.get("request_timeout_sec", 30))
MAX_RETRIES = int(CFG.get("max_retries", 4))
BACKOFF_BASE_MS = int(CFG.get("backoff_base_ms", 400))
BACKOFF_JITTER_MS = int(CFG.get("backoff_jitter_ms", 250))
RESPECT_RL = bool(CFG.get("respect_rate_limits", True))

# -------------------------------------------------------------------
# Tiny logging helpers
# -------------------------------------------------------------------
def log(msg: str) -> None:
    if VERBOSE:
        print(msg, flush=True)

def warn(msg: str) -> None:
    print(f"⚠️  {msg}", flush=True)

print("Using repo root:", REPO_ROOT)

# -------------------------------------------------------------------
# Canonical output layout helpers
# -------------------------------------------------------------------
def ensure_dir(p: pathlib.Path) -> None:
    p.mkdir(parents=True, exist_ok=True)

def repo_root() -> pathlib.Path:
    root = OUT_ROOT
    ensure_dir(root)
    return root

def tags_all_json() -> pathlib.Path:
    d = repo_root() / "tags"
    ensure_dir(d)
    return d / "tags.all.json"

def series_dir(kind: str, series: str) -> pathlib.Path:
    d = repo_root() / kind / series
    ensure_dir(d)
    return d

def pair_stem(base: str, compare: str) -> str:
    return f"{base}...{compare}"

def pair_json(series: str, stem: str, kind: str) -> pathlib.Path:
    # kind ∈ {"compare","commits","pulls","issues"}
    return series_dir(kind, series) / f"{stem}.{kind}.json"

def capsule_json(series: str, stem: str) -> pathlib.Path:
    return series_dir("commits_pr_issue", series) / f"{stem}.tarce_artifacts.json"

# -------------------------------------------------------------------
# JSON I/O
# -------------------------------------------------------------------
def read_json(path: pathlib.Path) -> Optional[Dict[str, Any]]:
    if not path.exists():
        return None
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def write_json(path: pathlib.Path, payload: Dict[str, Any]) -> None:
    ensure_dir(path.parent)
    if path.exists() and not OVERWRITE:
        log(f"Skip (exists): {path}")
        return
    tmp = path.with_suffix(path.suffix + ".tmp")
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    tmp.replace(path)
    log(f"✓ Wrote {path}")

# -------------------------------------------------------------------
# GitHub HTTP session + rate-limit aware helpers
# -------------------------------------------------------------------
SESSION = requests.Session()
SESSION.headers.update({
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
    "User-Agent": f"notebook-pipeline/{NAME}"
})

def _parse_reset_epoch(headers: Dict[str, Any]) -> Optional[int]:
    try:
        return int(headers.get("X-RateLimit-Reset") or headers.get("x-ratelimit-reset"))
    except Exception:
        return None

def _maybe_sleep_for_reset(resp: requests.Response) -> None:
    if not RESPECT_RL:
        return
    remaining = resp.headers.get("X-RateLimit-Remaining") or resp.headers.get("x-ratelimit-remaining")
    if remaining is not None and str(remaining).isdigit() and int(remaining) <= 0:
        reset_epoch = _parse_reset_epoch(resp.headers)
        if reset_epoch:
            now = int(time.time())
            delta = max(0, reset_epoch - now) + 1
            warn(f"Rate limit reached. Sleeping ~{delta}s until reset …")
            time.sleep(delta)

def _backoff_sleep(i: int) -> None:
    base = BACKOFF_BASE_MS / 1000.0
    jitter = random.uniform(0, BACKOFF_JITTER_MS / 1000.0)
    time.sleep((2 ** i) * base + jitter)

def rest_get_json(url: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """
    Generic GET with backoff + rate-limit handling.
    """
    last_err = None
    for i in range(MAX_RETRIES + 1):
        try:
            r = SESSION.get(url, params=params, timeout=REQ_TIMEOUT)
            # Honor 429 Retry-After if present
            if r.status_code == 429:
                ra = r.headers.get("Retry-After")
                if ra and ra.isdigit():
                    wait_s = int(ra)
                    warn(f"429 received. Sleeping {wait_s}s per Retry-After …")
                    time.sleep(wait_s)
                    continue
            if r.status_code >= 500:
                last_err = f"{r.status_code} {r.text[:200]}"
                _backoff_sleep(i)
                continue
            if r.status_code >= 400:
                raise RuntimeError(f"HTTP {r.status_code}: {r.text[:500]}")
            _maybe_sleep_for_reset(r)
            return r.json()
        except Exception as e:
            last_err = str(e)
            _backoff_sleep(i)
    raise RuntimeError(f"GET failed after retries: {url} :: {last_err}")

def gh_graphql(query: str, variables: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """
    GitHub GraphQL POST with backoff + rate-limit handling.
    Endpoint: https://api.github.com/graphql
    """
    url = "https://api.github.com/graphql"
    payload = {"query": query, "variables": variables or {}}
    last_err = None
    for i in range(MAX_RETRIES + 1):
        try:
            r = SESSION.post(url, json=payload, timeout=REQ_TIMEOUT)
            if r.status_code == 429:
                ra = r.headers.get("Retry-After")
                if ra and ra.isdigit():
                    wait_s = int(ra)
                    warn(f"429 received. Sleeping {wait_s}s per Retry-After …")
                    time.sleep(wait_s)
                    continue
            if r.status_code >= 500:
                last_err = f"{r.status_code} {r.text[:200]}"
                _backoff_sleep(i)
                continue
            if r.status_code >= 400:
                raise RuntimeError(f"GraphQL HTTP {r.status_code}: {r.text[:500]}")
            _maybe_sleep_for_reset(r)
            data = r.json()
            if "errors" in data:
                last_err = f"GraphQL errors: {data['errors']}"
                _backoff_sleep(i)
                continue
            return data
        except Exception as e:
            last_err = str(e)
            _backoff_sleep(i)
    raise RuntimeError(f"GraphQL failed after retries: {last_err}")

def rate_limit_snapshot() -> Dict[str, Any]:
    """
    Returns current REST rate-limit bucket (printed; not written to JSON outputs).
    """
    try:
        info = rest_get_json("https://api.github.com/rate_limit")
        core = info.get("resources", {}).get("core", {})
        remaining = core.get("remaining")
        limit = core.get("limit")
        reset = core.get("reset")
        when = datetime.fromtimestamp(reset, tz=timezone.utc).isoformat() if reset else None
        snap = {"limit": limit, "remaining": remaining, "reset_epoch": reset, "reset_iso": when}
        log(f"Rate limit: {remaining}/{limit}, resets at {when}")
        return snap
    except Exception as e:
        warn(f"Rate limit snapshot failed: {e}")
        return {}

# -------------------------------------------------------------------
# Light helpers for series + ordering
# -------------------------------------------------------------------
def semver_series(tag_name: str) -> str:
    """
    Extract 'vX.Y' series from tags like 'v4.2.3', '4.2.0-rc.1', etc.
    Falls back to 'v0.0' if not parseable.
    """
    m = re.search(r'v?(\d+)\.(\d+)', tag_name or "")
    if not m:
        return "v0.0"
    return f"v{int(m.group(1))}.{int(m.group(2))}"

def sorted_pairs_by_tag_time(pairs: List[Tuple[str, str]], tag_index: Dict[str, Dict[str, Any]]) -> List[Tuple[str, str]]:
    """
    Sort (base, compare) by tag timestamps ascending when both are tags;
    unknown timestamps sort last.
    """
    def ts(tag: str) -> float:
        rec = tag_index.get(tag)
        if rec and rec.get("tag_timestamp"):
            try:
                return datetime.fromisoformat(rec["tag_timestamp"].replace("Z", "+00:00")).timestamp()
            except Exception:
                pass
        return float("inf")
    return sorted(pairs, key=lambda bc: (ts(bc[0]), ts(bc[1])))

log("Bootstrap ready ✓")


Using repo root: /Users/rambodparsi/Desktop/OSS Repository Selection/web_scrapper/commit_pr_issue_analysis
Bootstrap ready ✓


### Config echo

In [22]:
print("Repo:", REPO)
print("Output root:", repo_root())
print("Overwrite:", OVERWRITE, "| Verbose:", VERBOSE)
rate_limit_snapshot()


Repo: mastodon/mastodon
Output root: mastodon
Overwrite: True | Verbose: True
Rate limit: 3791/5000, resets at 2025-10-26T14:16:07+00:00


{'limit': 5000,
 'remaining': 3791,
 'reset_epoch': 1761488167,
 'reset_iso': '2025-10-26T14:16:07+00:00'}

### Discover *.commits.json

In [23]:
from pathlib import Path

print("REPO:", REPO)
print("repo_root():", repo_root())

commits_base = Path(repo_root()) / "commits"
print("looking in:", commits_base)

commit_files = []
if commits_base.exists():
    commit_files = sorted(commits_base.rglob("*.commits.json"))

# Broad fallback if a previous run used a different out_root
if not commit_files:
    warn("No files under <name>/commits. Falling back to a broad search for */commits/**/*.commits.json …")
    commit_files = sorted(Path(REPO_ROOT).rglob("commits/**/*.commits.json"))

print("found BEFORE whitelist:", len(commit_files))
found_series = sorted({p.parent.name for p in commit_files})
print("series present on disk:", found_series)

SERIES_WHITELIST = CFG.get("series_whitelist") or []
if SERIES_WHITELIST:
    before = len(commit_files)
    commit_files = [p for p in commit_files if p.parent.name in set(SERIES_WHITELIST)]
    after = len(commit_files)
    print(f"applied whitelist {SERIES_WHITELIST} → {after}/{before} remain")

print("sample:")
for p in commit_files[:10]:
    print(" -", p)


REPO: mastodon/mastodon
repo_root(): mastodon
looking in: mastodon/commits
found BEFORE whitelist: 107
series present on disk: ['v4.0', 'v4.1', 'v4.2', 'v4.3', 'v4.4']
applied whitelist ['v4.0'] → 19/107 remain
sample:
 - mastodon/commits/v4.0/v4.0.0...v4.0.1.commits.json
 - mastodon/commits/v4.0/v4.0.0rc1...v4.0.0rc2.commits.json
 - mastodon/commits/v4.0/v4.0.0rc2...v4.0.0rc3.commits.json
 - mastodon/commits/v4.0/v4.0.0rc3...v4.0.0rc4.commits.json
 - mastodon/commits/v4.0/v4.0.0rc4...v4.0.0.commits.json
 - mastodon/commits/v4.0/v4.0.1...v4.0.2.commits.json
 - mastodon/commits/v4.0/v4.0.10...v4.0.11.commits.json
 - mastodon/commits/v4.0/v4.0.11...v4.0.12.commits.json
 - mastodon/commits/v4.0/v4.0.12...v4.0.13.commits.json
 - mastodon/commits/v4.0/v4.0.13...v4.0.14.commits.json


### Helpers: fetch PR (pulls API), labels (issues API), parse issue refs

In [24]:
import re
from typing import Dict, Any, List, Optional, Tuple

# --- PR details (Pulls API) ---
def fetch_pull(owner: str, name: str, number: int) -> Dict[str, Any]:
    url = f"https://api.github.com/repos/{owner}/{name}/pulls/{number}"
    return rest_get_json(url)  # state, merged_at, title, body, html_url, url
# Docs: GET /repos/{owner}/{repo}/pulls/{pull_number}

# --- Labels for a PR via Issues API (PRs are issues) ---
def fetch_issue_labels(owner: str, name: str, number: int) -> List[str]:
    url = f"https://api.github.com/repos/{owner}/{name}/issues/{number}"
    data = rest_get_json(url)  # returns labels for both Issues and PRs
    labs = data.get("labels") or []
    names = []
    for lab in labs:
        if isinstance(lab, dict):
            nm = lab.get("name")
            if nm: names.append(nm)
        elif isinstance(lab, str):
            names.append(lab)
    return names
# Docs: Labels endpoints live under the Issues API; PRs are issues.

# --- Closing keywords parser (keep as supplementary) ---
CLOSE_WORDS = (
    r"(?:close|closes|closed|fix|fixes|fixed|resolve|resolves|resolved)"
)
ISSUE_REF = r"(?:#(?P<num>\d+)|(?P<full>[A-Za-z0-9_.-]+/[A-Za-z0-9_.-]+#\d+))"

CLOSE_REGEX = re.compile(rf"\b{CLOSE_WORDS}\s+{ISSUE_REF}\b", re.IGNORECASE)
MENTION_REGEX = re.compile(rf"\b{ISSUE_REF}\b", re.IGNORECASE)

def extract_issue_refs_text(pr_title: str, pr_body: str) -> List[Dict[str, Any]]:
    """
    Text-based extraction; returns local #123 and cross-repo owner/repo#123.
    Relationship is decided later after merging with GraphQL results.
    """
    text = f"{pr_title or ''}\n\n{pr_body or ''}"

    closes = []
    for m in CLOSE_REGEX.finditer(text):
        if m.group("num"): closes.append(("#" + m.group("num")).lower())
        elif m.group("full"): closes.append(m.group("full").lower())

    mentions = []
    for m in MENTION_REGEX.finditer(text):
        if m.group("num"): mentions.append(("#" + m.group("num")).lower())
        elif m.group("full"): mentions.append(m.group("full").lower())

    # dedupe preserves order
    def uniq(seq):
        seen=set(); out=[]
        for s in seq:
            if s not in seen:
                seen.add(s); out.append(s)
        return out

    closes_u = set(uniq(closes))
    mentions_u = uniq(mentions)

    out = []
    for ref in mentions_u:
        # Mark as 'closes' if the keyword form was detected; otherwise 'references'.
        rel = "closes" if ref in closes_u else "references"
        if ref.startswith("#"):
            num = int(ref[1:])
            out.append({"number": num,
                        "html_url": f"https://github.com/{OWNER}/{NAME}/issues/{num}",
                        "relationship": rel})
        else:
            owner_repo, num = ref.split("#", 1)
            out.append({"number": int(num),
                        "html_url": f"https://github.com/{owner_repo}/issues/{num}",
                        "relationship": rel})
    return out
# Docs: supported closing keywords; can be in PR description or commit message.

# --- NEW: GraphQL PR.closingIssuesReferences (authoritative for “Linked issues”) ---
def gql_closing_issues_for_pr(owner: str, name: str, number: int) -> List[Dict[str, Any]]:
    """
    Returns issues that the PR *will close* on merge, as shown in the PR UI.
    Paginate with hasNextPage/endCursor.
    """
    query = """
    query($owner:String!, $name:String!, $number:Int!, $after:String) {
      repository(owner:$owner, name:$name) {
        pullRequest(number:$number) {
          closingIssuesReferences(first:100, after:$after) {
            pageInfo { hasNextPage endCursor }
            nodes {
              number
              url
            }
          }
        }
      }
    }"""
    vars = {"owner": owner, "name": name, "number": int(number), "after": None}
    results = []
    while True:
        data = gh_graphql(query, vars)
        pr = (((data.get("data") or {}).get("repository") or {}).get("pullRequest") or {})
        cir = pr.get("closingIssuesReferences") or {}
        for node in cir.get("nodes") or []:
            n = node.get("number")
            u = node.get("url")
            if isinstance(n, int) and u:
                results.append({"number": n, "html_url": u, "relationship": "closes"})
        pi = cir.get("pageInfo") or {}
        if not pi.get("hasNextPage"):
            break
        vars["after"] = pi.get("endCursor")
    return results
# Docs: PullRequest.closingIssuesReferences (matches “Linked issues” in UI).


### Build *.pulls.json for every pair

In [25]:
written = 0
skipped = 0

for cpath in commit_files:
    cpay = read_json(cpath)
    if not cpay:
        continue

    series = cpath.parent.name
    base = cpay.get("base")
    comp = cpay.get("compare")
    stem = pair_stem(base, comp)
    out_path = pair_json(series, stem, "pulls")

    if out_path.exists() and not OVERWRITE:
        log(f"Skip (exists): {out_path}")
        skipped += 1
        continue

    # Unique PR numbers referenced by commits in this pair
    prs_set = []
    seen = set()
    for it in cpay.get("items") or []:
        for r in it.get("pr_refs") or []:
            n = r.get("number")
            if isinstance(n, int) and n not in seen:
                seen.add(n); prs_set.append(n)

    pr_items = []
    for n in prs_set:
        try:
            pr = fetch_pull(OWNER, NAME, n)  # Pulls API (title/body/state/merged_at)
            pr_state  = pr.get("state")
            pr_title  = pr.get("title")
            pr_body   = pr.get("body")
            pr_html   = pr.get("html_url")
            pr_api    = pr.get("url")
            merged_at = pr.get("merged_at")

            # 1) authoritative “Linked issues” via GraphQL
            gql_refs = gql_closing_issues_for_pr(OWNER, NAME, n)

            # 2) supplementary text parsing (may include additional 'references')
            text_refs = extract_issue_refs_text(pr_title or "", pr_body or "")

            # 3) union: prefer GraphQL 'closes'; add extra text refs not present in gql
            key = lambda x: (x.get("html_url") or "").lower()
            gql_by_url = { key(x): x for x in gql_refs }
            merged_refs = list(gql_refs)  # start with authoritative closes

            for tr in text_refs:
                if key(tr) not in gql_by_url:
                    # leave its 'relationship' as parsed ('references' or 'closes' if text claimed so)
                    merged_refs.append(tr)

            # PR labels (Issues API, because PR is an issue)
            labels = fetch_issue_labels(OWNER, NAME, n)

            pr_items.append({
                "number": n,
                "html_url": pr_html,
                "api_url": pr_api,
                "state": pr_state,
                "merged_at": merged_at,
                "pr_title": pr_title,
                "pr_body": pr_body,
                "labels": labels,                 # (unchanged) list[str]
                "issue_refs": merged_refs         # now: GraphQL closes + extra references from text
            })

        except Exception as e:
            warn(f"PR #{n} fetch failed: {e}")
            pr_items.append({
                "number": n,
                "html_url": f"https://github.com/{OWNER}/{NAME}/pull/{n}",
                "api_url": f"https://api.github.com/repos/{OWNER}/{NAME}/pulls/{n}",
                "state": None,
                "merged_at": None,
                "pr_title": None,
                "pr_body": None,
                "labels": [],
                "issue_refs": []
            })

    payload = {
        "repo": REPO,
        "base": base,
        "compare": comp,
        "series": series,
        "items": pr_items
    }
    write_json(out_path, payload)
    written += 1

print(f"Done. Written: {written} | Skipped: {skipped}")
rate_limit_snapshot()


✓ Wrote mastodon/pulls/v4.0/v4.0.0...v4.0.1.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.0rc1...v4.0.0rc2.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.0rc2...v4.0.0rc3.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.0rc3...v4.0.0rc4.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.0rc4...v4.0.0.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.1...v4.0.2.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.10...v4.0.11.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.11...v4.0.12.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.12...v4.0.13.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.13...v4.0.14.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.14...v4.0.15.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.2...v4.0.3.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.3...v4.0.4.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.4...v4.0.5.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.5...v4.0.6.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.6...v4.0.7.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.7...v4.0.8.pulls.json
✓ Wrote mastodon/pulls/v4.0/v4.0.8

{'limit': 5000,
 'remaining': 3283,
 'reset_epoch': 1761488167,
 'reset_iso': '2025-10-26T14:16:07+00:00'}